# 🧬 P450 生成序列可视化分析（含关键残基统计、t-SNE 聚类、比对）

In [ ]:

import paddle
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.manifold import TSNE
import seaborn as sns
import numpy as np


## 📄 读取FASTA序列

In [ ]:

def read_sequences(filepath):
    sequences = []
    with open(filepath, "r") as f:
        seq = ""
        for line in f:
            if line.startswith(">"):
                if seq:
                    sequences.append(seq.replace("-", ""))
                seq = ""
            else:
                seq += line.strip()
        if seq:
            sequences.append(seq.replace("-", ""))
    return sequences

sequences = read_sequences("../data/gen_0.fasta")
print(f"共读取序列数: {len(sequences)}")


## 📊 序列长度分布

In [ ]:

lengths = [len(s) for s in sequences]
plt.figure(figsize=(8, 4))
plt.hist(lengths, bins=20, color='cornflowerblue', edgecolor='black')
plt.title("序列长度分布（去除填充符号 '-'）")
plt.xlabel("长度")
plt.ylabel("数量")
plt.grid(True)
plt.tight_layout()
plt.show()


## 🔤 氨基酸频率分布

In [ ]:

aa_counter = Counter("".join(sequences))
aa_freq_sorted = dict(sorted(aa_counter.items()))

plt.figure(figsize=(10, 4))
plt.bar(aa_freq_sorted.keys(), aa_freq_sorted.values(), color='salmon')
plt.title("氨基酸频率分布")
plt.xlabel("氨基酸")
plt.ylabel("出现次数")
plt.grid(True)
plt.tight_layout()
plt.show()


## ✅ 关键残基统计
我们检查生成序列是否在以下位置具有指定氨基酸：T114, F123, A220, M248, A317

In [ ]:

# 定义关键残基（假设这些位置固定）
key_positions = [113, 122, 219, 247, 316]
key_amino_acids = ['T', 'F', 'A', 'M', 'A']

def check_key_residues(seq):
    return all(seq[pos] == aa for pos, aa in zip(key_positions, key_amino_acids) if pos < len(seq))

valid_count = sum(check_key_residues(seq) for seq in sequences)
print(f"关键残基完全匹配的序列数: {valid_count} / {len(sequences)}")
print(f"匹配率: {valid_count / len(sequences) * 100:.2f}%")


## 🌐 t-SNE 聚类可视化
对蛋白序列进行one-hot编码后降维，展示分布情况

In [ ]:

# 氨基酸索引
AA = 'ACDEFGHIKLMNPQRSTVWY'
aa2idx = {aa: i for i, aa in enumerate(AA)}

def encode_sequence(seq, max_len=100):
    vec = np.zeros((max_len, len(AA)))
    for i, aa in enumerate(seq[:max_len]):
        if aa in aa2idx:
            vec[i, aa2idx[aa]] = 1
    return vec.flatten()

X = np.array([encode_sequence(seq) for seq in sequences])
X_embedded = TSNE(n_components=2, random_state=42, perplexity=30).fit_transform(X)

plt.figure(figsize=(6,6))
sns.scatterplot(x=X_embedded[:,0], y=X_embedded[:,1])
plt.title("t-SNE 蛋白序列聚类可视化")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.grid(True)
plt.show()


## 🧬 比对示意（生成序列 vs 假设F6H参考序列）

In [ ]:

ref = sequences[0]
for i in range(1, 4):
    test = sequences[i]
    print(f"--- 比对: 序列 {i} vs 参考 ---")
    print("参考: ", ref[:80])
    print("生成: ", test[:80])
    match = "".join(["|" if ref[j] == test[j] else "." for j in range(min(len(ref), len(test), 80))])
    print("匹配: ", match)
    print()
